# Ising model

# 1. Imports

In [1]:

# qiskit imports
from scipy.optimize import minimize
import numpy as np
# importing thermal state library and circuit from 
#from thermal_state import thermal_state
#from circuit import Circuit
import time
import matplotlib.pyplot as plt
import qiskit as qk
import qiskit.providers.aer.noise as noise
import sys
sys.path.append("..") # import one subdirectory up in files
#sys.path.append("..") # import one subdirectory up in files
#os.chdir("../holopy/")
import basic_circuits as circuits
from networks import IsoMPS
from isonetwork import QKParamCircuit, IsoTensor
import numpy as np 
import scipy.linalg as la 
import matplotlib.pyplot as plt

# tenpy imports
import tenpy as tp
from tenpy.tools.params import asConfig
from tenpy.networks.site import SpinHalfSite
from tenpy.networks.mps import MPS
from tenpy.models.model import CouplingMPOModel, NearestNeighborModel

# quspin imports
from __future__ import print_function, division
import sys,os
from quspin.operators import hamiltonian,quantum_operator # Hamiltonians and operators
from quspin.basis import spin_basis_1d # Hilbert space spin basis

# import thermal state
#from thermal_state import thermal_state 
from circuit_qubit1 import Circuit
from hamiltonian import model_mpo

# 2. Circuit structure

In [12]:

bd = 2 # should determine this before using
chimax = 2
d = 2
# classical circuit structure
def circuit(bd):
    """
    bd: bond dimension
    """
    # for bond dimension 2:
    if bd == 2:
        
        c = Circuit([("qubit", "p", d), ("qubit", "b", chimax)])
        def qub_x(params): return (np.pi/2, 0, params[0])
        def qub_y(params): return (np.pi/2, np.pi/2, params[0])
        def qub_z(params): return (0, 0, params[0])
        def qub_two(params): return (params[0])

        # one qubit rotation
        c.add_gate("rotation", qids = [0], n_params = 1, fn = qub_x)
        c.add_gate("rotation", qids = [0], n_params = 1, fn = qub_y)
        c.add_gate("rotation", qids = [0], n_params = 1, fn = qub_z)

        c.add_gate("rotation", qids = [1], n_params = 1, fn = qub_x)
        c.add_gate("rotation", qids = [1], n_params = 1, fn = qub_y)
        c.add_gate("rotation", qids = [1], n_params = 1, fn = qub_z)

        # XX rotation
        c.add_gate("XX", qids=[0, 1], n_params = 1, fn = qub_two)

        # YY rotation 
        c.add_gate("YY", qids=[0, 1], n_params = 1, fn = qub_two)    

        # ZZ rotation
        c.add_gate("ZZ", qids=[0, 1], n_params = 1, fn = qub_two)

        # one qubit rotation
        c.add_gate("rotation", qids = [0], n_params = 1, fn = qub_x)
        c.add_gate("rotation", qids = [0], n_params = 1, fn = qub_y)
        c.add_gate("rotation", qids = [0], n_params = 1, fn = qub_z)

        c.add_gate("rotation", qids = [1], n_params = 1, fn = qub_x)
        c.add_gate("rotation", qids = [1], n_params = 1, fn = qub_y)
        c.add_gate("rotation", qids = [1], n_params = 1, fn = qub_z)

        c.assemble()
    
    # for bond dimension 4:
    elif bd == 4:
        
        def qub_x(params): return (np.pi/2, 0, params[0])
        def qub_y(params): return (np.pi/2, np.pi/2, params[0])
        def qub_z(params): return (0, 0, params[0])
        def qub_two(params): return (params[0])
        #def qub_Rxy(params): return (params[0], params[1])
        #def qub_Rz(params): return (params[0])

        c = Circuit([("qubit", "p", d), ("qubit", "b", chimax), ("qubit", "b", chimax)])

        # first entangle the physical qubit with the first bond qubit
        c.add_gate("rotation", qids = [0], n_params = 1, fn = qub_x)
        c.add_gate("rotation", qids = [0], n_params = 1, fn = qub_y)
        c.add_gate("rotation", qids = [0], n_params = 1, fn = qub_z)

        c.add_gate("rotation", qids = [1], n_params = 1, fn = qub_x)
        c.add_gate("rotation", qids = [1], n_params = 1, fn = qub_y)
        c.add_gate("rotation", qids = [1], n_params = 1, fn = qub_z)

        # XX rotation
        c.add_gate("XX", qids=[0, 1], n_params = 1, fn = qub_two)

        # YY rotation 
        c.add_gate("YY", qids=[0, 1], n_params = 1, fn = qub_two)    

        # ZZ rotation
        c.add_gate("ZZ", qids=[0, 1], n_params = 1, fn = qub_two)

        # one qubit rotation
        c.add_gate("rotation", qids = [0], n_params = 1, fn = qub_x)
        c.add_gate("rotation", qids = [0], n_params = 1, fn = qub_y)
        c.add_gate("rotation", qids = [0], n_params = 1, fn = qub_z)

        c.add_gate("rotation", qids = [1], n_params = 1, fn = qub_x)
        c.add_gate("rotation", qids = [1], n_params = 1, fn = qub_y)
        c.add_gate("rotation", qids = [1], n_params = 1, fn = qub_z)

        # then do the same for the physical qubit and the second bond qubit

        # one qubit rotation
        c.add_gate("rotation", qids = [0], n_params = 1, fn = qub_x)
        c.add_gate("rotation", qids = [0], n_params = 1, fn = qub_y)
        c.add_gate("rotation", qids = [0], n_params = 1, fn = qub_z)

        c.add_gate("rotation", qids = [2], n_params = 1, fn = qub_x)
        c.add_gate("rotation", qids = [2], n_params = 1, fn = qub_y)
        c.add_gate("rotation", qids = [2], n_params = 1, fn = qub_z)

        # XX rotation
        c.add_gate("XX", qids=[0, 2], n_params = 1, fn = qub_two)

        # YY rotation 
        c.add_gate("YY", qids=[0, 2], n_params = 1, fn = qub_two)    

        # ZZ rotation
        c.add_gate("ZZ", qids=[0, 2], n_params = 1, fn = qub_two)

        # one qubit rotation
        c.add_gate("rotation", qids = [0], n_params = 1, fn = qub_x)
        c.add_gate("rotation", qids = [0], n_params = 1, fn = qub_y)
        c.add_gate("rotation", qids = [0], n_params = 1, fn = qub_z)

        c.add_gate("rotation", qids = [2], n_params = 1, fn = qub_x)
        c.add_gate("rotation", qids = [2], n_params = 1, fn = qub_y)
        c.add_gate("rotation", qids = [2], n_params = 1, fn = qub_z)

        c.assemble()   
    return c

c = circuit(bd)

In [11]:

# qiskit-based circuit structure
## circuit structure  using circuit_qubit ##
# circuit construction

d = 2
chimax = 2

def circuit(nb,params):
    backend = qk.Aer.get_backend('unitary_simulator')
    c = qk.QuantumCircuit(nb+1)
    circuits.add_su4_circ(c, 0, 1, params[:15]) 
    if nb ==2:
        circuits.add_su4_circ(c, 0, 2, params[15:]) 
    job = qk.execute(c, backend)
    result = job.result()
    #get the unitary matrix from the result object
    return result.get_unitary(c, decimals=8)

backend = qk.Aer.get_backend('qasm_simulator')
c = qk.QuantumCircuit(nb+1,2)
circuits.add_su4_circ(c, 1, 0, sweet_spot[2:]) 
c.draw('mpl')
c.measure(0,0)
job = qk.execute(c, backend,shots = 400)
result = job.result()
result.get_counts()

{'00': 304, '01': 96}

# Self-dual Ising model

# 3a. Energy 

In [3]:

# model params
#### for MPDO, N = number of sites. For iMPDO, for self_dual, set L = N = 3.
J = 1
V = 0.5
h = 1
l_uc = 1 # length of unit-cell
L = 3 # length of repetitions of unit-cell
N = L * l_uc # number of sites
nb = 1

In [4]:

# thermal-state-based Hamiltonian and boundary vects
Hamiltonian = model_mpo.sd_ising(J,V,h,N)
chi_H = Hamiltonian[0,:,0,0].size # size of Hamiltonian bond leg dimension
H_bvecr = None
H_bvecl = np.zeros(chi_H)
H_bvecl[-1] = 1
bdry_vecs2 = [H_bvecl,H_bvecr]

In [5]:

# DMRG calculations

"""Prototypical example of a quantum model: the transverse field Ising model.
Like the :class:`~tenpy.models.xxz_chain.XXZChain`, the transverse field ising chain
:class:`TFIChain` is contained in the more general :class:`~tenpy.models.spins.SpinChain`;
the idea is more to serve as a pedagogical example for a 'model'.
We choose the field along z to allow to conserve the parity, if desired.
"""
# Copyright 2018-2020 TeNPy Developers, GNU GPLv3
import numpy as np

__all__ = ['TFIModel', 'TFIChain']
class Self_Dual_Ising_Model(CouplingMPOModel):
    """Transverse field Ising model on a general lattice.
    The Hamiltonian reads:
    .. math ::
        H = - \sum_{\langle i,j\rangle, i < j} \mathtt{J} \sigma^x_i \sigma^x_{j}
            - \sum_{i} \mathtt{g} \sigma^z_i
    Here, :math:`\langle i,j \rangle, i< j` denotes nearest neighbor pairs, each pair appearing
    exactly once.
    All parameters are collected in a single dictionary `model_params`, which
    is turned into a :class:`~tenpy.tools.params.Config` object.
    Parameters
    ----------
    model_params : :class:`~tenpy.tools.params.Config`
        Parameters for the model. See :cfg:config:`TFIModel` below.
    Options
    -------
    .. cfg:config :: TFIModel
        :include: CouplingMPOModel
        conserve : None | 'parity'
            What should be conserved. See :class:`~tenpy.networks.Site.SpinHalfSite`.
        J, g : float | array
            Coupling as defined for the Hamiltonian above.
    """
    def init_sites(self, model_params):
        conserve = model_params.get('conserve', 'parity')
        assert conserve != 'Sz'
        if conserve == 'best':
            conserve = 'parity'
            if self.verbose >= 1.:
                print(self.name + ": set conserve to", conserve)
        site = SpinHalfSite(conserve=conserve)
        return site   
    def init_terms(self, model_params):
        J = np.asarray(model_params.get('J', 1.))
        g = np.asarray(model_params.get('V', 1.))
        h = np.asarray(model_params.get('h', 1,))
        for u in range(len(self.lat.unit_cell)):
            self.add_onsite(-h, u, 'Sigmax')
        for u1, u2, dx in self.lat.pairs['nearest_neighbors']:
            self.add_coupling(-J, u1, 'Sigmaz', u2, 'Sigmaz', dx)
        for u1, u2, dx in self.lat.pairs['next_nearest_neighbors']:
            self.add_coupling(V, u1, 'Sigmaz', u2, 'Sigmaz', dx)
        for u1, u2, dx in self.lat.pairs['nearest_neighbors']:
            self.add_coupling(V, u1, 'Sigmax', u2, 'Sigmax', dx)
        # done
        
class Self_Dual_Ising_Chain(Self_Dual_Ising_Model, CouplingMPOModel):
    """The :class:`TFIModel` on a Chain, suitable for TEBD.
    See the :class:`TFIModel` for the documentation of parameters.
    """
    def __init__(self, model_params):
        model_params = asConfig(model_params, self.__class__.__name__)
        model_params.setdefault('lattice', "Chain")
        CouplingMPOModel.__init__(self, model_params)
        
    def DMRG_Self_Dual_Ising_infinite(J,V,h,chi_max,bc,L,verbose=False):
        model_params = dict(L=L, J=J, V=V, h=h, bc_MPS=bc, conserve=None, verbose=verbose)
        M = Self_Dual_Ising_Chain(model_params)
        product_state = ["down"] * M.lat.N_sites
        psi = MPS.from_product_state(M.lat.mps_sites(), product_state, bc=M.lat.bc_MPS)
        dmrg_params = {
        'mixer': True,  # setting this to True helps to escape local minima
        'trunc_params': {
            'chi_max': chi_max,
            'svd_min': 1.e-6
        },
        'max_E_err': 1.e-6,
        'verbose': verbose,
         }
        # Sometimes, we want to call a 'DMRG engine' explicitly
        eng = tp.algorithms.dmrg.TwoSiteDMRGEngine(psi, M, dmrg_params)
        E, psi = eng.run()  # equivalent to dmrg.run() up to the return parameters.
        print("E = {E:.13f}".format(E=E))
        print("final bond dimensions: ", psi.chi)
        mag_x = np.mean(psi.expectation_value("Sigmax"))
        mag_z = np.mean(psi.expectation_value("Sigmaz"))
        print("<sigma_x> = {mag_x:.5f}".format(mag_x=mag_x))
        print("<sigma_z> = {mag_z:.5f}".format(mag_z=mag_z))
        #print("correlation length:", psi.correlation_length())
        if bc == 'infinite':
            E1 = E
        else:
            E1 = E/L   
        return E1
    
E_infinite = Self_Dual_Ising_Chain.DMRG_Self_Dual_Ising_infinite(J,V,h,chi_max = 2,
                                                                 bc='infinite',L=2,verbose=False)

E = -0.9084196159550
final bond dimensions:  [2, 2]
<sigma_x> = 0.30041
<sigma_z> = 0.27627


/home/shahin75/anaconda3/lib/python3.8/site-packages/tenpy/tools/params.py:225: UserWarning: unused option ['P_tol'] for config lanczos_params

  warnings.warn(msg.format(keys=sorted(unused), name=self.name))


In [8]:

## testing for impdo
# free energy 
T = 0
def circuit_qubit_free_energy(x,*args):
    #circuit = args[0]
    L = args[1]
    T = args[2]
    Hamiltonian = args[3]
    chi_H = args[4]
    bdry_vecs1 = args[5]
    bdry_vecs2 = args[6]
    l_uc = args[7]
    unitary_list = [circuit(nb,x[2*l_uc:]).reshape((2, 2**nb, 2, 2**nb))]
    F = thermal_state.free_energy(unitary_list,
                                  x,'density_matrix',
                                  L,Hamiltonian,
                                  T, chi_H,
                                  bdry_vecs1,bdry_vecs2,
                                  'method_I',None) 
    return F

 
bdry_vecs1 = None
#### bdry_vecs1 = TBD for MPDO, bdry_vecs1 = [None,None] and for iMPDO, bdry_vecs1 = None
args = (c,L,T,Hamiltonian,chi_H,bdry_vecs1,bdry_vecs2,l_uc)
rng = np.random.default_rng()
x0 = rng.uniform(high=2*np.pi,size = nb*15 + 2)
result = minimize(circuit_qubit_free_energy,x0=x0,args=args,method = 'nelder-mead')
sweet_spot = result.x
free_energy = result.fun
print(f'Free energy = {free_energy}, T = {T}')
print(f'DMRG estimate (infinite length): {E_infinite}')

Free energy = -0.9087211472053924, T = 0
DMRG estimate (infinite length): -0.9084196159549833


# Transverse-field Ising model

# 3b. Energy 

In [9]:

# model params
#### for MPDO, N = number of sites. For iMPDO, for tfim, set L = N = 2.
J = 1
h = 1
g = 0
l_uc = 1 # length of unit-cell
L = 2 # length of repetitions of unit-cell
N = L * l_uc
nb = 1 # number of sites

In [10]:

# thermal-state-based Hamiltonian and boundary vects
Hamiltonian = model_mpo.tfim_ising(J,g,h,N)
chi_H = Hamiltonian[0,:,0,0].size # size of Hamiltonian bond leg dimension
H_bvecr = None
H_bvecl = np.zeros(chi_H)
H_bvecl[-1] = 1
bdry_vecs2 = [H_bvecl,H_bvecr]

In [12]:

# DMRG calculations

"""Prototypical example of a quantum model: the transverse field Ising model.
Like the :class:`~tenpy.models.xxz_chain.XXZChain`, the transverse field ising chain
:class:`TFIChain` is contained in the more general :class:`~tenpy.models.spins.SpinChain`;
the idea is more to serve as a pedagogical example for a 'model'.
We choose the field along z to allow to conserve the parity, if desired.
"""
# Copyright 2018-2020 TeNPy Developers, GNU GPLv3

__all__ = ['TFIModel', 'TFIChain']


class TFIZModel(CouplingMPOModel):
    r"""Transverse field Ising model on a general lattice.
    The Hamiltonian reads:
    .. math ::
        H = - \sum_{\langle i,j\rangle, i < j} \mathtt{J} \sigma^x_i \sigma^x_{j}
            - \sum_{i} \mathtt{g} \sigma^z_i
    Here, :math:`\langle i,j \rangle, i< j` denotes nearest neighbor pairs, each pair appearing
    exactly once.
    All parameters are collected in a single dictionary `model_params`, which
    is turned into a :class:`~tenpy.tools.params.Config` object.
    Parameters
    ----------
    model_params : :class:`~tenpy.tools.params.Config`
        Parameters for the model. See :cfg:config:`TFIModel` below.
    Options
    -------
    .. cfg:config :: TFIModel
        :include: CouplingMPOModel
        conserve : None | 'parity'
            What should be conserved. See :class:`~tenpy.networks.Site.SpinHalfSite`.
        J, g : float | array
            Coupling as defined for the Hamiltonian above.
    """
    def init_sites(self, model_params):
        conserve = model_params.get('conserve', 'parity')
        assert conserve != 'Sz'
        if conserve == 'best':
            conserve = 'parity'
            if self.verbose >= 1.:
                print(self.name + ": set conserve to", conserve)
        site = SpinHalfSite(conserve=conserve)
        return site

    def init_terms(self, model_params):
        J = np.asarray(model_params.get('J', 1.))
        g = np.asarray(model_params.get('g', 1.))
        h = np.asarray(model_params.get('h', 1.))
        for u in range(len(self.lat.unit_cell)):
            self.add_onsite(-g, u, 'Sigmaz')
        for u in range(len(self.lat.unit_cell)):
            self.add_onsite(-h, u, 'Sigmax')
        for u1, u2, dx in self.lat.pairs['nearest_neighbors']:
            self.add_coupling(-J, u1, 'Sigmaz', u2, 'Sigmaz', dx)
        # done

class TFIZChain(TFIZModel, NearestNeighborModel):
    """The :class:`TFIModel` on a Chain, suitable for TEBD.
    See the :class:`TFIModel` for the documentation of parameters.
    """
    def __init__(self, model_params):
        model_params = asConfig(model_params, self.__class__.__name__)
        model_params.setdefault('lattice', "Chain")
        CouplingMPOModel.__init__(self, model_params)
        
def DMRG_tfim(J,g,h,chi_max,bc,L,verbose=False):
    model_params = dict(L=L, J=J, g=g, h=h, bc_MPS=bc, conserve=None, verbose=verbose)
    M = TFIZChain(model_params)
    product_state = ["down"] * M.lat.N_sites
    psi = MPS.from_product_state(M.lat.mps_sites(), product_state, bc=M.lat.bc_MPS)
    dmrg_params = {
        'mixer': True,  # setting this to True helps to escape local minima
        'trunc_params': {
            'chi_max': chi_max,
            'svd_min': 1.e-6
        },
        'max_E_err': 1.e-6,
        'verbose': verbose,
    }
    # Sometimes, we want to call a 'DMRG engine' explicitly
    eng = tp.algorithms.dmrg.TwoSiteDMRGEngine(psi, M, dmrg_params)
    E, psi = eng.run()  # equivalent to dmrg.run() up to the return parameters.
    print("E = {E:.13f}".format(E=E))
    print("final bond dimensions: ", psi.chi)
    mag_x = np.mean(psi.expectation_value("Sigmax"))
    mag_z = np.mean(psi.expectation_value("Sigmaz"))
    print("<sigma_x> = {mag_x:.5f}".format(mag_x=mag_x))
    print("<sigma_z> = {mag_z:.5f}".format(mag_z=mag_z))
    #print("correlation length:", psi.correlation_length())
    if bc == 'infinite':
        E1 = E
    else:
        E1 = E/L
    return E1

E_infinite = DMRG_tfim(J,g,h,chi_max = 2,bc='infinite',L=2,verbose=True)

Reading 'bc_MPS'='infinite' for config TFIZChain
Reading 'conserve'=None for config TFIZChain
Reading 'L'=2 for config TFIZChain
Reading 'J'=1 for config TFIZChain
Reading 'g'=0 for config TFIZChain
Reading 'h'=1 for config TFIZChain
Subconfig 'trunc_params'=Config(<3 options>, 'trunc_params') for config TwoSiteDMRGEngine
Initial sweeps...
Updating environment
.
Reading 'max_E_err'=1e-06 for config TwoSiteDMRGEngine
Reading 'mixer'=True for config TwoSiteDMRGEngine
Updating environment
.....
sweep 10, age = 128
Energy = -1.2725208311389054, S = nan, norm_err = 6.0e-09
Current memory usage 325.0 MB, time elapsed: 1.4 s
Delta E = nan, Delta S = 0.0000e+00 (per sweep)
max_trunc_err = 1.3780e-04, max_E_trunc = 3.5078e-04
MPS bond dimensions: [2, 2]
disable mixer after 15 sweeps, final amplitude 3.05e-10
Updating environment
.....
sweep 20, age = 248
Energy = -1.2725205336563619, S = 0.1224680200039622, norm_err = 5.4e-09
Current memory usage 325.0 MB, time elapsed: 2.3 s
Delta E = 2.9748e-

/home/shahin75/anaconda3/lib/python3.8/site-packages/tenpy/tools/params.py:225: UserWarning: unused option ['P_tol'] for config lanczos_params

  warnings.warn(msg.format(keys=sorted(unused), name=self.name))


In [13]:

## testing for impdo
# free energy 
T = 0
def circuit_qubit_free_energy(x,*args):
    #circuit = args[0]
    L = args[1]
    T = args[2]
    Hamiltonian = args[3]
    chi_H = args[4]
    bdry_vecs1 = args[5]
    bdry_vecs2 = args[6]
    l_uc = args[7]
    unitary_list = [circuit(nb,x[2*l_uc:]).reshape((2, 2**nb, 2, 2**nb))]
    F = thermal_state.free_energy(unitary_list,
                                  x,'density_matrix',
                                  L,Hamiltonian,
                                  T, chi_H,
                                  bdry_vecs1,bdry_vecs2,
                                  'method_I',None) 
    return F
 
bdry_vecs1 = None
#### bdry_vecs1 = TBD for MPDO, bdry_vecs1 = [None,None] and for iMPDO, bdry_vecs1 = None
args = (c,L,T,Hamiltonian,chi_H,bdry_vecs1,bdry_vecs2,l_uc)
rng = np.random.default_rng()
x0 = rng.uniform(high=2*np.pi,size = nb*15 + 2)
result = minimize(circuit_qubit_free_energy,x0=x0,args=args,method = 'nelder-mead')
sweet_spot = result.x
free_energy = result.fun
print(f'Free energy = {free_energy}, T = {T}')
print(f'DMRG estimate (infinite length): {E_infinite}')

Free energy = -1.2715659135564217, T = 0
DMRG estimate (infinite length): -1.2725205334309564
